In [ ]:
from main import *

: 

In [ ]:
train_csv = "./data/train.csv"
test_csv = "./data/test.csv"
train_new_csv = "./data/train_new.csv"
test_new_csv = "./data/test_new.csv"

import pandas as pd

train_data = pd.read_csv(train_csv)
test_data = pd.read_csv(test_csv)
train_new_data = pd.read_csv(train_new_csv)
test_new_data = pd.read_csv(test_new_csv)

train_data_new = pd.concat([train_data, train_new_data], axis = 1)
test_data_new = pd.concat([test_data, test_new_data], axis = 1)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split

train_X = train_data.drop(columns = ["Y1", "Y2"])
test_X = test_data.drop(columns = ["id"])

train_X_new = train_data_new.drop(columns = ["Y1", "Y2"])
test_X_new = test_data_new.drop(columns = ["id"])

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", KernelPCA(kernel="rbf"))
])

pipe_new = Pipeline([
    ("imputer", IterativeImputer(max_iter = 5)),
    ("scaler", StandardScaler()),
    ("pca", KernelPCA(kernel = "rbf"))
])

train_X = pipe.fit_transform(train_X)
test_X = pipe.transform(test_X)

train_X_new = pipe_new.fit_transform(train_X_new)
test_X_new = pipe_new.transform(test_X_new)

train_X = pd.DataFrame(train_X, columns = [f"{i}" for i in range(len(train_X[0]))])
test_X = pd.DataFrame(test_X, columns = [f"{i}" for i in range(len(test_X[0]))])

train_X_new = pd.DataFrame(train_X_new, columns = [f"{i}" for i in range(len(train_X_new[0]))])
test_X_new = pd.DataFrame(test_X_new, columns = [f"{i}" for i in range(len(test_X_new[0]))])

# train_X, val_X, train_y, val_y = train_test_split(
#     train_X, train_data[["Y1", "Y2"]],
#     test_size = .2
# )

# train_X_new, val_X_new, train_y_new, val_y_new = train_test_split(
#     train_X_new, train_data_new[["Y1", "Y2"]],
#     test_size = .2
# )

train_data = pd.concat([train_X, train_data[["Y1", "Y2"]]], axis = 1)
test_data = pd.concat([test_data["id"], test_X], axis = 1)

train_data_new = pd.concat([train_X_new, train_data_new[["Y1", "Y2"]]], axis = 1)
test_data_new = pd.concat([test_data_new["id"], test_X_new], axis = 1)

In [ ]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

lgbm = MultiOutputRegressor(
    lgb.LGBMRegressor(
        n_estimators=200,
        learning_rate=.1,
        #early_stopping_rounds=50,
        n_jobs=1,
        verbose=-1
    )
)

tscv = TimeSeriesSplit(n_splits = 10)
scores = cross_val_score(lgbm, train_X_new, train_data_new[["Y1", "Y2"]], cv = tscv, scoring = "r2")
print(scores.mean())
print(scores.std())
print(scores)

0.6391277829774683
0.07885049499961867
[0.64879527 0.63318229 0.71487874 0.44125953 0.65085359 0.66221906
 0.69496637 0.67392403 0.55771874 0.7134802 ]
